# Notes
CORELS is a great way to test the value-add of other ML models.  Generally speaking, CORELS builds an optimal one-sided decision tree, giving simple rules that are highly predictive.  That, of course, has lots of advantages over more complex models, and can help us understand feature impotance, patterns in the data, etc. 

Further (limited) documentation can be found at https://pycorels.readthedocs.io/en/latest/

## The inputs should be:
- the file that we want to learn from

## The outputs should be: 
- a list of rules that predict re-arrest. 

As you can see below, though, we first have to bin & dummy all the continuous variables. It's a fairly quick win if you want to start here. 

In [ ]:
import pickle
with open("30_window-1000_consumers-3_depth.dat", "rb") as f: 
    X_train, X_test, y_train, y_test = pickle.load(f) 
    df = X_train.append(X_test)
df['class'] = y_train.append(y_test)
my_vars = df.columns
for var in my_vars:
    if df[var].dtype == "float64":
        print(var)

In [2]:
# here is where you would bin each of the floats. 
# Gender, obviously, can be turned into a binary. 
# Maybe try 5 or 10 bins for the others? 

In [3]:
import pandas as pd
import numpy as np
num_bins= 5                           #main issue here is num_bins to set the binning you want.. number is inclusive [1:X]
bin_labels = np.arange(0,num_bins)+1  #passing this so labeling is 1:X and not 0:(X-1)

In [4]:
for var in my_vars:
    if df[var].dtype == "float64":
        if var == "consumer_gender":
            df[var] = pd.cut(df[var],2,labels=[0,1])
        else:
            df[var] = pd.cut(df[var],num_bins,labels=bin_labels)

In [5]:
df["class"] = pd.cut(df["class"],2,labels=[0,1])

Group_cutoff is here in case you would want to organize number of events into a [a,b,c,..., (> max_val)] type set, instead of re-binning the range

This is an optional change and does uniquely modify this col compared to the others

`df["COUNT(events)"].value_counts()` is a great way to get a sense of the values

In [6]:
def group_cutoff(x, max_val):
    if x > max_val:
        x = max_val
    return x

df["COUNT(events)"] = df["COUNT(events)"].apply(lambda x: group_cutoff(x, 10)) # could be achieved with `.apply(group_cutoff, args=[10])` as well

In [12]:
# re-bins all the fields that are larger sets than num_bins..
# astype('category') is set to all cols so that pd.get_dummies() behaves
my_vars1 = df.columns
for var in my_vars1:
    if df[var].nunique() > num_bins:
        df[var] = pd.cut(df[var],num_bins,labels=bin_labels)
    df[var] = df[var].astype('category')

In [8]:
df2 = pd.get_dummies(df)
df2.drop("class_0", axis=1, inplace=True)

In [9]:
df2.to_csv("arrests.csv",index=False)

In [10]:
import csv
import numpy as np
fname = "arrests.csv"
features = []
prediction_name = ""

with open(fname, "r") as f:
    features = f.readline().strip().split(",")
    prediction_name = features[-1]
    features = features[0:-1]

data = np.genfromtxt(fname, dtype=np.uint8, skip_header=1, delimiter=",")

X = data[:, 0:-1]
y = data[:, -1]


In [11]:
from corels import *

# Create the model, with 10000 as the maximum number of iterations 
c = CorelsClassifier(n_iter=10000)

# Fit, and score the model on the training set
a = c.fit(X, y).score(X, y)

# Print the model's accuracy on the training set
print(a)


RULELIST:
prediction = False
0.8827142857142857


## Bonus investigation:

(This section assumes model sensitivity = specificity = accuracy)

Based on the population split, as defined in our prediction column, we use bayes-theorem to predict the likelihood that a user identified by a fictional model is actually a member of the group we wish to identify

When the model accuracy is 50/50, the bayes value reduces to the population split ratio, so we plot up from there. 

The point at which we equally select target-group vs not-target-group is when model accuracy matches the not-target-group population ratio (in this case 1-y.mean())

This is useful for estimating cost range and effectiveness - e.g. if you had a model with 92% accuracy, based on a (.8827/.1173) population split.. you serve 92% of your target group, but only 60% of the people you serve with interventions are within the target group; 40% of those identified and served were, in theory, unneccessary

This highlights the value of increasingly accurate models - while you might get to 95% and say 'plenty close,' 30% of those identified by this model in our earlier population split are still not in the target group. Further accuracy may not add many people served from the target group, but it will reduce the overall number of interventions needed to do so

In [83]:
model_acc = 0.95 #fictional model accuracy
target_users = y.mean() # with the binary list the % True is the mean 
def bayes_test(model_acc, target_users):
    return (model_acc*target_users) / ((model_acc*target_users) + ((1-model_acc)*(1-target_users)))

plot_dict = {}
for i in range(500,1000,5):
    i = i/1000
    plot_dict[i] = bayes_test(i,target_users)

df_toplot = pd.DataFrame.from_dict(plot_dict, orient='index', columns=["Chance Identified User is in Target Group"])

In [ ]:
%matplotlib inline
df_toplot.plot()